Librerias

In [76]:
import os
from dotenv import load_dotenv
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
from tqdm import tqdm
from pathlib import Path


## Extract

In [3]:
load_dotenv()

login =  os.getenv('JUMPSELLER_LOGIN')
authtoken =  os.getenv('JUMPSELLER_AUTHTOKEN')

In [77]:
def obtener_datos(endpoint, login, authtoken):
    # Asegurar que la carpeta exista
    Path("data/raw").mkdir(parents=True, exist_ok=True)


    datos = []
    page = 1
    barra= tqdm(desc=f"Descargando '{endpoint}'", unit=" páginas")

    while (True):
        url = f"https://api.jumpseller.com/v1/{endpoint}.json?page={page}&limit=50"
        r = requests.get(url, auth=HTTPBasicAuth(login, authtoken))
        if r.status_code != 200:
            print(f"❌ Error al obtener {endpoint}:", r.status_code)
            break

        data = r.json()
        if not data:
            break

        datos.extend(data)
        barra.update(1) # avanza la barra en 1 unidad
        page += 1

    barra.close()


    print(f"✅ Data obtenida: {endpoint.title()}. Total páginas {page-1}")
    datos= pd.json_normalize(datos)
    datos.to_parquet(f"data/raw/{endpoint}_raw.parquet", index=False) # se almacena la data raw
    return datos


In [ ]:
df_products = obtener_datos("products", login, authtoken)


Descargando 'products': 23 páginas [00:28,  1.25s/ páginas]


✅ Data obtenida: Products. Total páginas 23


In [79]:
df_customers = obtener_datos("customers", login, authtoken)
df_orders = obtener_datos("orders", login, authtoken)


Descargando 'customers': 31 páginas [00:13,  2.29 páginas/s]


✅ Data obtenida: Customers. Total páginas 31


Descargando 'orders': 29 páginas [00:19,  1.48 páginas/s]


✅ Data obtenida: Orders. Total páginas 29


## Transform

In [27]:
df_products.head(5)

,product.id,product.name,product.page_title,product.description,product.meta_description,product.price,product.cost_per_item,product.compare_at_price,product.weight,product.stock,...,product.height,product.diameter,product.google_product_category,product.categories,product.images,product.variants,product.fields,product.permalink,product.discount,product.currency
0,27766653,Cámara Canon Mirrorless EOS R8 Body,Cámara Canon Mirrorless EOS R8 Body,"<div class=""product-description""><h1>\n ...",Canon EOS R8: Desempeño excepcional y...,1999990.0,NaN,2499990.0,1.4,1,...,20.0,0.0,141,"[{'id': 1992493, 'name': 'Video y Fotografía',...","[{'id': 57589702, 'url': 'https://images.jumps...",[],"[{'id': 13528130, 'custom_field_id': 68122, 't...",5803c002aa,0.0,CLP
1,27766654,Cámara Canon Mirrorless EOS R6 MKII Body,Cámara Canon Mirrorless EOS R6 MKII Body,"<div class=""product-description""><h1>Canon EOS...",Canon EOS R6 Mark II: Desempeño multimedia exc...,2989990.0,NaN,NaN,1.5,1,...,20.0,0.0,141,"[{'id': 1992493, 'name': 'Video y Fotografía',...","[{'id': 57590020, 'url': 'https://images.jumps...",[],"[{'id': 13528139, 'custom_field_id': 68122, 't...",5666c003aa,0.0,CLP
2,27766655,Cámara Canon Mirrorless EOS R50 Body,Cámara Canon Mirrorless EOS R50 Body,"<div class=""product-description""><h1>Canon EOS...",Canon EOS R50: Cámara Mirrorless compacta y ve...,1999990.0,NaN,24999990.0,1.2,0,...,20.0,0.0,141,"[{'id': 1992493, 'name': 'Video y Fotografía',...","[{'id': 57590079, 'url': 'https://images.jumps...",[],"[{'id': 13528148, 'custom_field_id': 68122, 't...",5811c002aa,0.0,CLP
3,27766656,Cámara Canon Mirrorless EOS R7 - Body,Cámara Canon Mirrorless EOS R7 - Body,"<div class=""product-description""><h1>\n ...",Canon EOS R7: Desempeño de alta veloc...,1549990.0,NaN,NaN,1.4,0,...,20.0,0.0,141,"[{'id': 1992493, 'name': 'Video y Fotografía',...","[{'id': 57590211, 'url': 'https://images.jumps...",[],"[{'id': 13528157, 'custom_field_id': 68122, 't...",5137c002,0.0,CLP
4,27766657,Cámara Canon Mirrorless EOS R10 - Body,Cámara Canon Mirrorless EOS R10 - Body,"<div class=""product-description""><h1>Canon EOS...",Canon EOS R10: Cámara Mirrorless compacta y po...,994990.0,NaN,1149990.0,1.3,1,...,20.0,0.0,141,"[{'id': 1992493, 'name': 'Video y Fotografía',...","[{'id': 57590336, 'url': 'https://images.jumps...",[],"[{'id': 13528166, 'custom_field_id': 68122, 't...",5331c002,0.0,CLP


In [ ]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1137 entries, 0 to 1136
Data columns (total 37 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   product.id                       1137 non-null   int64  
 1   product.name                     1137 non-null   object 
 2   product.page_title               1137 non-null   object 
 3   product.description              1132 non-null   object 
 4   product.meta_description         1132 non-null   object 
 5   product.price                    1137 non-null   float64
 6   product.cost_per_item            486 non-null    float64
 7   product.compare_at_price         84 non-null     float64
 8   product.weight                   1137 non-null   float64
 9   product.stock                    1137 non-null   int64  
 10  product.stock_unlimited          1137 non-null   bool   
 11  product.stock_threshold          1137 non-null   int64  
 12  product.stock_notifi

In [57]:
df_products['type'].unique()

array(['physical'], dtype=object)

columnas relevantes
- id, name, price, stock, stock_notification, brand, created_at, update_at, categories

In [50]:
df_products.columns

Index(['id', 'name', 'page_title', 'description', 'meta_description', 'price',
       'cost_per_item', 'compare_at_price', 'weight', 'stock',
       'stock_unlimited', 'stock_threshold', 'stock_notification', 'sku',
       'brand', 'barcode', 'featured', 'reviews_enabled', 'status',
       'shipping_required', 'type', 'days_to_expire', 'created_at',
       'updated_at', 'package_format', 'length', 'width', 'height', 'diameter',
       'google_product_category', 'categories', 'images', 'variants', 'fields',
       'permalink', 'discount', 'currency'],
      dtype='object')

In [58]:
columnas_necesarias = ['id', 'name', 'brand', 'price', 'created_at', 'updated_at', 'stock', 'stock_notification', 'categories']
df = df_products[columnas_necesarias].copy()

In [60]:
df.isnull().sum()

id                    0
name                  0
brand                 1
price                 0
created_at            0
updated_at            0
stock                 0
stock_notification    0
categories            0
dtype: int64

In [63]:
type(df['categories'][0])
df_prueba= df.copy()
df_prueba = pd.json_normalize(df['categories'])


In [68]:
df.head(1)

,id,name,brand,price,created_at,updated_at,stock,stock_notification,categories
0,27766653,Cámara Canon Mirrorless EOS R8 Body,Canon,1999990.0,2024-11-23 00:20:22 UTC,2025-08-12 00:47:37 UTC,1,True,"[{'id': 1992493, 'name': 'Video y Fotografía',..."


In [72]:
categorias= df['categories'][0]
lista_categorias= []
for categoria in categorias:
    nombre_cate= categoria['name']
    lista_categorias.append(nombre_cate)

In [73]:
lista_categorias

['Video y Fotografía',
 'Cámaras',
 'Audio',
 'Micrófonos',
 'Marcas',
 'Canon',
 'Micrófono para Vlogging',
 'Cámaras Mirrorless',
 'Cámaras Semiprofesionales',
 'Cámaras para Vlog',
 'Cámaras para Streaming',
 'Cámaras para Youtuber',
 'Cámaras para TikTok',
 'Cámaras para Instagram',
 'Cámaras para Podcast',
 'Cámaras para Grabar Contenido',
 'Cámaras para Transmitir en Vivo',
 'Cámaras para Transmitir en Vivo por Facebook',
 'Cámaras para Viajes',
 'Cámaras para Fotografiar Aves']

In [ ]:
import pandas as pd

def limpiar_y_preparar_productos(df):
    # Renombrar
    df.columns = df.columns.str.replace('^product\.', '', regex=True)

    # Columnas clave
    columnas_necesarias = ['id', 'name', 'brand', 'price', 'created_at', 'updated_at', 'stock', 'stock_notification', 'categories']
    columnas_disponibles = [col for col in columnas_necesarias if col in df.columns]
    df = df[columnas_disponibles].copy()

    # Fechas a datetime
    for fecha_col in ['created_at', 'updated_at']:
        if fecha_col in df.columns:
            df[fecha_col] = pd.to_datetime(df[fecha_col], errors='coerce')
            nulos_fecha = df[fecha_col].isna().sum()
            if nulos_fecha > 0:
                print(f"⚠️ {nulos_fecha} valores nulos en {fecha_col} tras conversión a datetime.")

    # Nulos en texto
    for col in ['name', 'brand']:
        if col in df.columns:
            df[col] = df[col].fillna('No especificado')

    # Nulos en numéricos
    if 'price' in df.columns:
        df['price'] = pd.to_numeric(df['price'], errors='coerce').fillna(0)

    print("✅ Limpieza y preparación completada.")
    return df



Celular Samusng A55
[{'id': 13528220, 'custom_field_id': 68122, 'type': 'selection', 'label': 'Tamaño del sensor', 'value_id': 137278, 'value': 'APS-C', 'variant_id': None}, {'id': 13528221, 'custom_field_id': 68123, 'type': 'selection', 'label': 'Entrada Micrófono', 'value_id': 137281, 'value': 'Sí', 'variant_id': None}, {'id': 13528222, 'custom_field_id': 68124, 'type': 'selection', 'label': 'Salida de Audífonos', 'value_id': 137282, 'value': 'Sí', 'variant_id': None}, {'id': 13528223, 'custom_field_id': 68125, 'type': 'selection', 'label': 'Megapíxeles', 'value_id': 139859, 'value': '24.2 MP', 'variant_id': None}, {'id': 13528224, 'custom_field_id': 68126, 'type': 'selection', 'label': 'Resolución de Video', 'value_id': 139860, 'value': '4K a 60p', 'variant_id': None}, {'id': 13528225, 'custom_field_id': 68127, 'type': 'selection', 'label': 'Pantalla', 'value_id': 139861, 'value': 'Táctil articulada', 'variant_id': None}, {'id': 13528226, 'custom_field_id': 66671, 'type': 'text', 'l